# 📊 8.3 — Laboratorio: Dashboard Interactivo de Ventas

En este laboratorio construiremos un **dashboard analítico de ventas** utilizando **Plotly Express** y **ipywidgets**.

💡 *Objetivo:* combinar los conocimientos de Pandas y visualización para crear un panel interactivo que permita explorar las ventas por fecha, producto y región (simulada).

In [2]:
!pip install ipywidgets


In [25]:
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

print('✅ Notebook 8.3 — Dashboard Interactivo cargado correctamente.')

✅ Notebook 8.3 — Dashboard Interactivo cargado correctamente.


---
## 1️⃣ Cargar y preparar datos

Usaremos el dataset `ventas.csv` y añadiremos una columna `region` para poder segmentar las visualizaciones.

In [26]:
df = pd.read_csv('../../datasets/ventas.csv')
df.rename(columns={'cantidad': 'unidades', 'precio': 'precio_unitario'}, inplace=True)
df['fecha'] = pd.to_datetime(df['fecha'])
df['ingresos'] = df['unidades'] * df['precio_unitario']

# Asignar regiones simuladas
regiones = ['Norte', 'Sur', 'Este', 'Oeste']
df['region'] = np.random.choice(regiones, len(df))

df.head()

,fecha,producto,unidades,precio_unitario,ingresos,region
0,2025-10-01,A,10,5.2,52.0,Este
1,2025-10-01,B,4,7.1,28.4,Oeste
2,2025-10-01,C,8,4.0,32.0,Norte
3,2025-10-01,D,3,6.5,19.5,Oeste
4,2025-10-02,A,6,5.0,30.0,Este


---
## 2️⃣ Crear controles interactivos (filtros)

Usaremos **ipywidgets** para permitir la selección de producto y rango de fechas.

In [27]:
import ipywidgets
ipywidgets.__version__


'8.1.8'

In [ ]:
# Selector de producto
productos = df['producto'].unique().tolist()
selector_producto = widgets.SelectMultiple(
    options=productos,
    value=[productos[0]],
    description='Producto:',
    style={'description_width': 'initial'}
)

# Selector de rango de fechas
rango_fechas = widgets.SelectionRangeSlider(
    options=sorted(df['fecha'].dt.strftime('%Y-%m-%d').unique()),
    index=(0, len(df['fecha'].unique()) - 1),
    description='Rango de fechas:',
    layout={'width': '600px'}
)

display(selector_producto, rango_fechas)

SelectMultiple(description='Producto:', index=(0,), options=('A', 'B', 'C', 'D'), style=DescriptionStyle(descr…

SelectionRangeSlider(description='Rango de fechas:', index=(0, 5), layout=Layout(width='600px'), options=('202…

---
## 3️⃣ Función de actualización del dashboard

La función `actualizar_dashboard` filtrará los datos y generará 3 gráficos:

- 📈 Evolución temporal de ingresos
- 🧭 Ingresos por región
- 🍰 Distribución por producto

In [34]:
def actualizar_dashboard(productos, rango):
    fecha_inicio, fecha_fin = pd.to_datetime(rango)
    df_filtrado = df[
        (df['producto'].isin(productos)) &
        (df['fecha'] >= fecha_inicio) & (df['fecha'] <= fecha_fin)
    ]
    

    resumen(df_filtrado)
    
    # Gráfico 1 — Ingresos por fecha
    ingresos_dia = df_filtrado.groupby('fecha', as_index=False)['ingresos'].sum()
    fig1 = px.line(ingresos_dia, x='fecha', y='ingresos', title='Evolución de Ingresos', markers=True)
    fig1.update_layout(template='plotly_white')
    
    # Gráfico 2 — Ingresos por región
    ingresos_region = df_filtrado.groupby('region', as_index=False)['ingresos'].sum()
    fig2 = px.bar(ingresos_region, x='region', y='ingresos', title='Ingresos por Región', color='region',
                  color_discrete_sequence=px.colors.qualitative.Vivid)
    fig2.update_layout(template='plotly_white')
    
    # Gráfico 3 — Distribución por producto
    ingresos_producto = df_filtrado.groupby('producto', as_index=False)['ingresos'].sum()
    fig3 = px.pie(ingresos_producto, values='ingresos', names='producto',
                  title='Participación por Producto', color_discrete_sequence=px.colors.qualitative.Pastel)
    fig3.update_traces(textinfo='percent+label')
    
    fig1.show()
    fig2.show()
    fig3.show()


---
## 4️⃣ Conectar widgets con el dashboard

Usaremos `widgets.interactive` para vincular la función de actualización con los filtros definidos.

In [35]:
widgets.interactive(
    actualizar_dashboard,
    productos=selector_producto,
    rango=rango_fechas
)

interactive(children=(SelectMultiple(description='Producto:', index=(2,), options=('A', 'B', 'C', 'D'), style=…

---
## 5️⃣ 🧩 Ejercicio — Métricas adicionales

Añade debajo del dashboard un **resumen numérico** con:

- Total de ingresos
- Promedio de unidades vendidas
- Producto más vendido (por ingresos)

👉 *Tip:* puedes usar `df_filtrado.agg()` y `idxmax()`.

In [ ]:
# ✏️ Tu código aquí...

### ✅ Solución propuesta

In [31]:
def resumen(df_filtrado):
    total_ingresos = df_filtrado['ingresos'].sum()
    promedio_unidades = df_filtrado['unidades'].mean().round(2)
    producto_top = df_filtrado.groupby('producto')['ingresos'].sum().idxmax()
    print(f"💰 Total ingresos: {total_ingresos:.2f} €")
    print(f"📦 Promedio de unidades: {promedio_unidades}")
    print(f"🏆 Producto más vendido: {producto_top}")

---
## 6️⃣ Exportar Dashboard

Puedes exportar cualquiera de los gráficos en formato HTML interactivo o imagen.

In [ ]:
# fig1.write_html('dashboard_ingresos.html')
# print('✅ Dashboard exportado como dashboard_ingresos.html')

In [36]:
from ipywidgets.embed import embed_minimal_html
import ipywidgets as widgets

# Tu dashboard interactivo
dashboard = widgets.interactive(
    actualizar_dashboard,
    productos=selector_producto,
    rango=rango_fechas
)

# Exportar solo el widget interactivo a un HTML
embed_minimal_html(
    'dashboard_interactivo.html',
    views=[dashboard],
    title='Dashboard Interactivo'
)

dashboard


interactive(children=(SelectMultiple(description='Producto:', index=(2,), options=('A', 'B', 'C', 'D'), style=…

In [38]:
import panel as pn

df = pd.DataFrame({
    "fecha": pd.date_range("2025-10-01", periods=10, freq="D"),
    "A": [10,20,25,30,28,40,35,33,38,42],
    "B": [5,7,9,12,10,8,15,11,14,16],
    "C": [30,32,35,40,39,41,45,47,46,49],
    "D": [50,48,47,55,52,58,57,60,59,62]
})


In [39]:
selector = pn.widgets.Select(
    name="Producto",
    options=["A","B","C","D"],
    value = "A"
)

In [40]:
rango = pn.widgets.DateRangeSlider(
    name="Rango de fechas",
    start=df["fecha"].min(),
    end=df["fecha"].max(),
    value=(df["fecha"].min(), df["fecha"].max())
)

In [ ]:



def actualizar_dashboard_panel(producto, rango):
    fecha_ini, fecha_fin = rango
    fecha_ini = pd.to_datetime(fecha_ini)
    fecha_fin = pd.to_datetime(fecha_fin)


    df_filtrado = df[(df["fecha"] >= fecha_ini) & (df["fecha"] <= fecha_fin)]
 

    fig1 = px.line(df_filtrado, x="fecha", y=producto)
    fig2 = px.bar(df_filtrado, x="fecha", y=producto)

    return pn.Column(pn.pane.Plotly(fig1), pn.pane.Plotly(fig2))




In [44]:
dashboard = pn.bind(actualizar_dashboard_panel, producto=selector, rango=rango)

panel_dashboard = pn.Column(
    "BLABLABLA",
    selector,
    rango,
    dashboard
)

In [45]:
panel_dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'035c023f-7fb2-4103-80c3-b41f7da30fac': {'version…

In [46]:
panel_dashboard.save("salida.hmtl", embed=True)

---
## 7️⃣ 🧠 Conclusiones

- Los widgets permiten interactividad directa con los datos.
- Plotly facilita la creación de visualizaciones profesionales.
- Este enfoque es ideal para construir dashboards ligeros en notebooks sin necesidad de frameworks adicionales.